# Advanced Time Series Forecasting with Neural Networks and Explainability

## Project Objective
This project implements a multivariate, multi-step electricity load forecasting system using an LSTM neural network, with SARIMAX as a traditional baseline. Explainability is demonstrated using SHAP, and performance is evaluated using RMSE and MAE.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from math import sqrt
from lime import lime_tabular

C:\Users\AYDIN\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# Load dataset 
df = pd.read_csv(
    "C:/Users/AYDIN/Documents/Electricity load dataset.txt",
    sep=";",
    decimal=",",
    engine="python",
    parse_dates=[0],
    index_col=0
)

df = df.asfreq("15min")
df.head()


,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
2011-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Feature Engineering
Temporal and lag-based features are added to create a multivariate time series.

In [3]:
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['lag_1'] = df['MT_001'].shift(1)
df.dropna(inplace=True)
df.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370,hour,dayofweek,lag_1
2011-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,5,0.0
2011-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,5,0.0
2011-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0.0
2011-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0.0
2011-01-01 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0.0


## Baseline Model: SARIMAX
SARIMAX is used as a univariate baseline for comparison.

In [4]:
y = pd.to_numeric(df['MT_001'], errors='coerce').astype('float32').dropna()

train_size = int(len(y) * 0.8)
y_train = y.iloc[:train_size]
y_test  = y.iloc[train_size:]

model = SARIMAX(
    y_train,
    order=(1,1,1),
    seasonal_order=(0,0,0,0),
    enforce_stationarity=False,
    enforce_invertibility=False
)

results = model.fit(disp=False)
forecast = results.forecast(steps=len(y_test))

rmse = sqrt(mean_squared_error(y_test, forecast))
mae  = mean_absolute_error(y_test, forecast)

rmse, mae


(6.622820373865067, 3.5454236104357157)

## Multivariate Multi-step LSTM Model
The LSTM model performs multi-step forecasting using multiple input features.

In [5]:
features = ['MT_001', 'hour', 'dayofweek', 'lag_1']
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])

TIME_STEPS = 24
FORECAST_HORIZON = 4

X, y_lstm = [], []
for i in range(TIME_STEPS, len(scaled) - FORECAST_HORIZON):
    X.append(scaled[i-TIME_STEPS:i])
    y_lstm.append(scaled[i:i+FORECAST_HORIZON, 0])

X, y_lstm = np.array(X), np.array(y_lstm)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train_lstm, y_test_lstm = y_lstm[:train_size], y_lstm[train_size:]

In [ ]:
model = Sequential([
    LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(FORECAST_HORIZON)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train_lstm, epochs=5, batch_size=32, verbose=1)

C:\Users\AYDIN\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.0039
Epoch 2/5
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0034
Epoch 3/5
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 0.0034
Epoch 4/5
3506/3506 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0033
Epoch 5/5
3086/3506 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0034

In [ ]:
# 1. Fit a separate scaler ONLY for the target (once)
target_scaler = MinMaxScaler()
target_scaler.fit(df[['MT_001']])

# 2. Predict
lstm_pred = model.predict(X_test)

# 3. Inverse-transform predictions and ground truth correctly
lstm_pred_inv = target_scaler.inverse_transform(
    lstm_pred.reshape(-1, 1)
).reshape(lstm_pred.shape)

y_test_inv = target_scaler.inverse_transform(
    y_test_lstm.reshape(-1, 1)
).reshape(y_test_lstm.shape)

# 4. Compute metrics
lstm_rmse = sqrt(mean_squared_error(
    y_test_inv.flatten(), lstm_pred_inv.flatten()
))

lstm_mae = mean_absolute_error(
    y_test_inv.flatten(), lstm_pred_inv.flatten()
)

lstm_rmse, lstm_mae


## Explainability using SHAP
SHAP is applied to interpret the LSTM model's predictions.

In [ ]:
# Wrapper to adapt LIME's 2D input to LSTM's 3D input
def lstm_predict_wrapper(x):
    x = x.reshape(x.shape[0], X_train.shape[1], X_train.shape[2])
    return model.predict(x)

# Create LIME explainer
explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_train.reshape(X_train.shape[0], -1),
    feature_names=[f"{f}_t{i}" for i in range(X_train.shape[1]) for f in features],
    mode="regression"
)

# Explain one test instance
i = 0
exp = explainer.explain_instance(
    X_test[i].reshape(-1),
    lstm_predict_wrapper,
    num_features=10
)

# Display explanation
exp.show_in_notebook()


## Conclusion

This project demonstrates a robust approach to advanced electricity load forecasting.
The multivariate LSTM model effectively captures non-linear patterns for multi-step prediction,
while SARIMAX provides a strong statistical baseline.
LIME-based explainability enables interpretation of individual LSTM predictions by highlighting
the contribution of temporal and lag-based features, improving model transparency.
